In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Mise en place de la dataframe GeoPandas
import math
import random
import time
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import requests
import io
from geopy.geocoders import Nominatim

In [3]:
json_url = "https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/data-es/exports/json?lang=fr&timezone=Europe%2FParis&select=equip_nom%2C%20reg_nom%2C%20equip_service_date%2C%20dep_nom%2C%20equip_type_name%2C%20coordonnees%2C%20equip_x%2C%20equip_y"
req = requests.get(json_url)
json_data = req.json()
df_dirty = pd.DataFrame(json_data)
df_dirty['equip_service_date'] = pd.to_numeric(df_dirty['equip_service_date'], errors='coerce')
df = df_dirty.dropna(subset=['equip_service_date'])

# Les coordonnées sont au format WGS84 d'après la documentation. On définit donc le CRS en adéquation.
geodf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['equip_x'], df['equip_y']), crs="EPSG:4326")  

In [4]:
#Test
geodf.head(10)

,equip_nom,reg_nom,equip_service_date,dep_nom,equip_type_name,coordonnees,equip_x,equip_y,geometry
3,STAND DE TIR,Provence-Alpes-Côte d'Azur,1986.0,Bouches-du-Rhône,Pas de tir à la cible,"{'lon': 5.58084, 'lat': 43.3852}",5.580840,43.385200,POINT (5.58084 43.3852)
5,TERRAIN DE BOULES 2,Provence-Alpes-Côte d'Azur,1994.0,Bouches-du-Rhône,Terrain de boules,"{'lon': 5.5817, 'lat': 43.64302}",5.581700,43.643020,POINT (5.5817 43.64302)
6,AVEN DE JESUS CHRIST,Provence-Alpes-Côte d'Azur,1948.0,Bouches-du-Rhône,Site de spéléologie sportive et/ou éducative,"{'lon': 5.4619, 'lat': 43.37887}",5.461900,43.378870,POINT (5.4619 43.37887)
7,MUR D'ESCALADE,Provence-Alpes-Côte d'Azur,2005.0,Bouches-du-Rhône,Structure Artificielle d'Escalade,"{'lon': 5.46891, 'lat': 43.3533}",5.468910,43.353300,POINT (5.46891 43.3533)
8,SALLE DE DANSE,Provence-Alpes-Côte d'Azur,2004.0,Bouches-du-Rhône,Salle de danse,"{'lon': 5.46891, 'lat': 43.3533}",5.468910,43.353300,POINT (5.46891 43.3533)
9,STADE DE FOOT,Provence-Alpes-Côte d'Azur,2012.0,Bouches-du-Rhône,Terrain de football,"{'lon': 5.46757, 'lat': 43.35455}",5.467570,43.354550,POINT (5.46757 43.35455)
16,CENTRE EQUESTRE LA MARQUISANE,Provence-Alpes-Côte d'Azur,1998.0,Bouches-du-Rhône,Carrière,"{'lon': 5.4709, 'lat': 43.35531}",5.470900,43.355310,POINT (5.4709 43.35531)
18,SALLE DE DANSE,Provence-Alpes-Côte d'Azur,2013.0,Bouches-du-Rhône,Salle de danse,"{'lon': 5.00246, 'lat': 43.81332}",5.002460,43.813320,POINT (5.00246 43.81332)
19,SALLE DE MUSCULATION,Provence-Alpes-Côte d'Azur,2013.0,Bouches-du-Rhône,Salle d'haltérophilie,"{'lon': 5.002458, 'lat': 43.813324}",5.002458,43.813324,POINT (5.00246 43.81332)
20,MUR D'ESCALADE,Provence-Alpes-Côte d'Azur,2013.0,Bouches-du-Rhône,Structure Artificielle d'Escalade,"{'lon': 5.002458, 'lat': 43.813324}",5.002458,43.813324,POINT (5.00246 43.81332)


In [5]:
#Recherche de l'infrastructure la plus proche de (x,y) en exploitant les opérations vectorisées de Pandas :
def closest_infra(x,y, gdf) :
    point = Point(x,y) # Coord. en format WGS84
    gdf_proj = gdf.to_crs(epsg=3857) 
    point_proj = gpd.GeoSeries([point], crs="EPSG:4326").to_crs(epsg=3857)[0] # Reprojeter les coordonnées (format WGS84) en système métrique pour augmenter la précision
    gdf_proj['distance'] = gdf_proj.geometry.distance(point_proj) # Calcul des distances entre notre point et toutes les infrastructures, grâce à GeoPandas
    closest_id = gdf_proj['distance'].idxmin() # Retourner l'indice du minimul avec idxmin
    dist = gdf_proj.loc[closest_id, 'distance']
    return gdf.loc[closest_id], dist

In [6]:
closest_infra(2.207237, 48.711014, geodf) # Coordonnées approximatives de l'ENSAE. On retrouve l'écurie de l'X, juste en face :)

(equip_nom                                        MANEGE
 reg_nom                                   Île-de-France
 equip_service_date                               1976.0
 dep_nom                                         Essonne
 equip_type_name                                  Manège
 coordonnees           {'lon': 2.20645, 'lat': 48.70994}
 equip_x                                         2.20645
 equip_y                                        48.70994
 geometry                       POINT (2.20645 48.70994)
 Name: 235696, dtype: object,
 np.float64(201.25366060421027))

In [7]:
closest_infra(1.422434324, 45.54397854, geodf) # Point généré aléatoirement, près de Limoges

(equip_nom                                BASE NAUTIQUE
 reg_nom                             Nouvelle-Aquitaine
 equip_service_date                              1989.0
 dep_nom                                   Haute-Vienne
 equip_type_name                  Stade de ski nautique
 coordonnees           {'lon': 1.42864, 'lat': 45.5458}
 equip_x                                        1.42864
 equip_y                                        45.5458
 geometry                       POINT (1.42864 45.5458)
 Name: 265918, dtype: object,
 np.float64(749.0279346331382))

In [8]:
def closest_infra_address(address, gdf):
    geolocator = Nominatim(user_agent="projet_python_infra_sport") #Géocodage de l'adresse
    location = geolocator.geocode(address)
    #
    if location is None:
        return("Adresse introuvable")
    #
    input_point = Point(location.longitude, location.latitude) #Récupération des coordonnées géographiques
    input_gdf = gpd.GeoDataFrame(geometry=[input_point], crs="EPSG:4326")
    #
    gdf_proj = gdf.to_crs("EPSG:2154") # Projection de gdf en système métrique, base temporaire (pour obtenir la distance en mètres)
    input_gdf_proj = input_gdf.to_crs("EPSG:2154") # Projection des coordonnées géographiques en système métrique
    input_point_proj = input_gdf_proj.geometry.iloc[0] # Extraction du point reprojeté en système métrique
    #
    gdf_proj['distance'] = gdf_proj.geometry.distance(input_point_proj) # Calcul des distances en système métrique
    closest_id = gdf_proj['distance'].idxmin() # Retourner l'indice du minimul avec idxmin
    closest = gdf.loc[closest_id] # Obtenir l'infrastructure la plus proche dans la base principale
    dist = gdf_proj.loc[closest_id, 'distance'] # Obtenir la distance
    return closest, dist, "Distance en m : {}".format(dist)

In [9]:
closest_infra_address('5 Av. Le Chatelier, Palaiseau', geodf) # Infrastructure la plus proche de l'ENSAE (écurie de l'X !)

(equip_nom                                        MANEGE
 reg_nom                                   Île-de-France
 equip_service_date                               1976.0
 dep_nom                                         Essonne
 equip_type_name                                  Manège
 coordonnees           {'lon': 2.20645, 'lat': 48.70994}
 equip_x                                         2.20645
 equip_y                                        48.70994
 geometry                       POINT (2.20645 48.70994)
 Name: 235696, dtype: object,
 np.float64(159.05219064268366),
 'Distance en m : 159.05219064268366')

In [10]:
# NB : essayez de copier les coordonnées sur Google Maps en les intervertissant (Google Maps lit les coordonnées sous le format (lat, lon) et non (lon, lat))

In [11]:
def closest_infra_address_equiptype(address, equip_type, gdf):
    gdf_filter = gdf[gdf['equip_type_name'] == equip_type]
    gdf_filter = gdf_filter.copy()
    return closest_infra_address(address, gdf_filter)

In [12]:
closest_infra_address_equiptype('5 Av. Le Chatelier, Palaiseau', 'Court de tennis', geodf) 

(equip_nom                              COURTS DE TENNIS 4
 reg_nom                                     Île-de-France
 equip_service_date                                 1964.0
 dep_nom                                           Essonne
 equip_type_name                           Court de tennis
 coordonnees           {'lon': 2.196922, 'lat': 48.700712}
 equip_x                                          2.196922
 equip_y                                         48.700712
 geometry                       POINT (2.196922 48.700712)
 Name: 235651, dtype: object,
 np.float64(1400.1613311646356),
 'Distance en m : 1400.1613311646356')

In [13]:
def generate_random_coord() :
    lat_min, lat_max = 43.0, 50.0 # Limites de latitude de la France métropolitaine
    lon_min, lon_max = -1.0, 7.0 # Limites de longitude de la France métropolitaine
    random_lat = random.uniform(lat_min, lat_max)
    random_lon = random.uniform(lon_min, lon_max) # Génération de coordonnées aléatoires 
    return random_lon, random_lat

In [14]:
def generate_random_address():
    lat_min, lat_max = 41.0, 51.0 # Limites de latitude de la France métropolitaine (grossièrement définies, approximées par un rectangle)
    lon_min, lon_max = -5.0, 9.0 # Limites de longitude de la France métropolitaine (grossièrement définies, approximées par un rectangle)
    random_lat = random.uniform(lat_min, lat_max)
    random_lon = random.uniform(lon_min, lon_max) # Génération de coordonnées aléatoires 
    geolocator = Nominatim(user_agent="projet_python_infra_sport_adress")
    location = geolocator.reverse((random_lat, random_lon), exactly_one=True) # Transformation des coordonnées en adresse postale
    if location and location.address: # Vérification de l'existence de l'adresse
        return location.address
    else:
        return "Adresse introuvable"

In [15]:
def mean_dist(n, gdf) :
    somme = 0
    for _ in range(n) :
        x,y = generate_random_coord()
        if isinstance(closest_infra(x,y, gdf)[1], float): # Permet de s'assurer que la distance est bien un float
            somme+=closest_infra(x,y, gdf)[1]
    return somme/n

In [16]:
start = time.time()
m = mean_dist(100, geodf)
end = time.time()
m, end - start # Temps estimé pour n = 1000 : 541 secondes, soit 9min

(np.float64(4286.256196170753), 40.529632568359375)

In [18]:
for _ in range(3) :
    print(closest_infra_address(generate_random_address(), geodf))
    time.sleep(1)

(equip_nom                           TERRAIN DE FOOTBALL
reg_nom                              Nouvelle-Aquitaine
equip_service_date                               1965.0
dep_nom                                    Haute-Vienne
equip_type_name                     Terrain de football
coordonnees           {'lon': 0.851927, 'lat': 45.7007}
equip_x                                        0.851927
equip_y                                         45.7007
geometry                       POINT (0.851927 45.7007)
Name: 270004, dtype: object, np.float64(1220.6858198897128), 'Distance en m : 1220.6858198897128')
(equip_nom                                             Salle polyvalente
reg_nom                                                       Normandie
equip_service_date                                               1991.0
dep_nom                                                  Seine-Maritime
equip_type_name       Salles polyvalentes / des fêtes / non spéciali...
coordonnees                        